<a href="https://colab.research.google.com/github/lgelbman/QuantitativeAnalysisProject/blob/main/QuantitativeAnalysisProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Distributions:

###Claim Severity Distribution:

Gamma distribution with $\mu = 4$, $ \sigma = 6$.

$\mu = \frac{\alpha}{\lambda} = 4$.

$\sigma = 6 \rightarrow \sigma^2 = (\frac{\alpha}{\lambda^2}) = 36$.

$\longrightarrow \alpha = \frac{4}{9} , \lambda = \frac{1}{9}$.

###Claim Frequency Distribution:

Poisson distribution with ${\mu} = {\lambda} = 5$.



##Default Parameters:

In [12]:
#gamma
alpha = 4/9
beta = 1/9   #(named beta because lambda is a keyword in python)

#poisson
mu = 5

##These sliders can be used to set the R.V. parameters (they are only visible on Colab not on github): 

0 means default parameters (alpha = 4/9, lambda = 1/9 for the gamma and 5 for the poisson).

Please run this code cell (or run all) after modifying the parameters.

In [13]:
gamma_alpha_slider = 0 #@param {type:"slider", min:0, max:100, step:0.1}
gamma_lambda_slider = 0 #@param {type:"slider", min:0, max:100, step:0.1}
poisson_lambda_slider = 0 #@param {type:"slider", min:0, max:100, step:0.1}

if gamma_alpha_slider != 0:
  alpha = gamma_alpha_slider

if gamma_lambda_slider != 0:
  beta = gamma_lambda_slider

if poisson_lambda_slider != 0:
  mu = poisson_lambda_slider

In [14]:
#parameters for Panjer's Formula
a = 0
b = beta

##Import Statements:

In [15]:
import math
from scipy.stats import gamma
import numpy as np
import pandas as pd

##Discretized Gamma:

This function takes a natural number n, and returns the probability that a gamma(alpha, lambda) RV is in the range (n - .5, n + .5).

In [16]:
def discretized_gamma(x):
  # the scale parameter in this library is the reciprocal of beta or lambda. Source: https://stackoverflow.com/a/42151088
  gamma_rv = gamma(a=alpha, scale=(1/beta))
  return gamma_rv.cdf(x + .5) - gamma_rv.cdf(x - .5)

##Panjer's Formula:

$P_N (z)=e^{(\lambda(z-1))} =e^{(5(z-1))}$

$g_x=\frac{1}{(1-af_0 )}   \sum_{y=1}^{x} (a+\frac{b}{x} y)  f_y g_{(x-y)}$

$g_0 = P_N (f_0 ) = e^{(\lambda(f_0-1))} $

$poisson \rightarrow a = 0, b = {\mu} = {\lambda} = 5$

$g_x=\sum_{y=1}^{x} \frac{5}{x} y f_y g_{(x-y)}$

$g_0 = P_N (f_0 ) = e^{(5(f_0-1))}$





##Panjer's Recursive Formula First Attempt:

This version cannot efficiently calculate more than the first few probabilities. ( $O(n!)$ running time)

In [17]:
def get_compound_probability(x):

  #base case: total = 0.
  if x == 0:
    return get_probability_that_total_is_zero()
  
  #recursive case:
  sum = 0
  for y in range(1, x + 1):
    sum += ((b/x) * y) * discretized_gamma(y) * get_compound_probability(x - y)
  return sum


#This "helper function" handles the base case for both versions:
def get_probability_that_total_is_zero():

  def evaluate_poisson_pgf(z):
    return math.exp(mu * (z - 1))

  f_0 = discretized_gamma(0)
  result = evaluate_poisson_pgf(f_0)
  return result

##Panjer's Recursive Formula Memoized Version:

This version uses a dictionary to store previously computed probabilities so they are not recomputed.

In [18]:
probabilities = {}

def memoized_compound_probability(x):
  if x in probabilities:
    return probabilities[x] 
  elif x == 0:
    p = get_probability_that_total_is_zero()
    probabilities[x] = p
    return p
  else:
    sum = 0
    for y in range(1, x + 1):
      sum += ((b/x) * y) * discretized_gamma(y) * memoized_compound_probability(x - y)
    probabilities[x] = sum
    return sum

##Calculating the first 40 probabilities:

In [19]:
probabilities = {}

for i in range(40):
  probability = memoized_compound_probability(i)
  probabilities[i] = probability
  print("probability of {} = {}".format(i, probability))


probability of 0 = 0.03130569408169159
probability of 1 = 0.0006159429765646497
probability of 2 = 0.0003686404901306715
probability of 3 = 0.00026461969841395304
probability of 4 = 0.0002031463124096832
probability of 5 = 0.0001616301978332766
probability of 6 = 0.0001314983732497827
probability of 7 = 0.00010862849822556142
probability of 8 = 9.073753617295115e-05
probability of 9 = 7.643529550028071e-05
probability of 10 = 6.481561597053002e-05
probability of 11 = 5.525690685109715e-05
probability of 12 = 4.7315615594892976e-05
probability of 13 = 4.066519310678742e-05
probability of 14 = 3.505910356049508e-05
probability of 15 = 3.0307349333624615e-05
probability of 16 = 2.6260988907147623e-05
probability of 17 = 2.2801588327344982e-05
probability of 18 = 1.9833830963459766e-05
probability of 19 = 1.7280213788005423e-05
probability of 20 = 1.5077160502472405e-05
probability of 21 = 1.3172120285168385e-05
probability of 22 = 1.1521366998252343e-05
probability of 23 = 1.0088305758861